# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib pandas

In [1]:
import cv2
import numpy as np
import os
%matplotlib nbagg
import matplotlib.pyplot as plt
import time
import mediapipe as mp
from PIL import Image
from IPython.display import display
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# 2. Keypoints using MP holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image =  cv2.cvtColor(image, cv2.COLOR_BGR2RGB)# COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image,results):
   
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image,results):
   
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=1))
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

# 3. Extract keypoint values

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)

    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

Setup folders for collection

In [7]:
DATA_PATH = os.path.join('BL_Data')
actions = np.array(['Ajebaje', 'Akash', 'Alada', 'Allah', 'Asha', 'Bakko', 'Bank', 'Bari', 'Bebsha', 'Bepar', 'Beyam', 'Bhromon', 'Bibaho', 'Biggan', 'Biruddhe', 'Bisoy', 'Boi', 'Boka', 'Bristi', 'Camera', 'Cha', 'Chaowa', 'Churanto', 'Dam', 'Daraw', 'Dawat', 'Dharona', 'Dhowa', 'Dokandar', 'Dol', 'Dowa_Kora', 'Druto', 'Dupur', 'Durgondho', 'Ful', 'Gari', 'Ghi', 'Ghori', 'Ghosito_Howa', 'Ghumano', 'Haat', 'Hashi', 'Hassokor', 'Injection', 'Jailkhana', 'Jinish', 'Jogajog', 'Kachi', 'Kapor', 'Kashi', 'Khaowa', 'Khoma', 'Klanto', 'Kukur', 'Mach', 'Matha', 'Matha_Betha', 'Mongol', 'Moyla', 'Name', 'Norachora', 'Ojon', 'Onusoron', 'Opomanjonok', 'Osushtho', 'Oushodh', 'Petuk', 'Phone', 'Pochondo', 'Porikkha', 'Poriskar', 'Prostut', 'Protarona', 'Raat', 'Rajdhani', 'Rasta', 'Sabdhan', 'Sajano', 'Shasti', 'Shokti', 'Shorto', 'Shotru', 'Sokal', 'Soman', 'Somossha', 'Somoy', 'Songbad', 'Sonkirno', 'Sosta', 'Table', 'Taka', 'Tamasha', 'Tapmatra', 'Tarikh', 'Touri_Kora', 'Tumi', 'Unnoto', 'Upor', 'Vaggo', 'Valo', 'Vari', 'Vule_jawa'])


no_sequences = 30
sequence_length = 30

In [8]:
actions

array(['Ajebaje', 'Akash', 'Alada', 'Allah', 'Asha', 'Bakko', 'Bank',
       'Bari', 'Bebsha', 'Bepar', 'Beyam', 'Bhromon', 'Bibaho', 'Biggan',
       'Biruddhe', 'Bisoy', 'Boi', 'Boka', 'Bristi', 'Camera', 'Cha',
       'Chaowa', 'Churanto', 'Dam', 'Daraw', 'Dawat', 'Dharona', 'Dhowa',
       'Dokandar', 'Dol', 'Dowa_Kora', 'Druto', 'Dupur', 'Durgondho',
       'Ful', 'Gari', 'Ghi', 'Ghori', 'Ghosito_Howa', 'Ghumano', 'Haat',
       'Hashi', 'Hassokor', 'Injection', 'Jailkhana', 'Jinish', 'Jogajog',
       'Kachi', 'Kapor', 'Kashi', 'Khaowa', 'Khoma', 'Klanto', 'Kukur',
       'Mach', 'Matha', 'Matha_Betha', 'Mongol', 'Moyla', 'Name',
       'Norachora', 'Ojon', 'Onusoron', 'Opomanjonok', 'Osushtho',
       'Oushodh', 'Petuk', 'Phone', 'Pochondo', 'Porikkha', 'Poriskar',
       'Prostut', 'Protarona', 'Raat', 'Rajdhani', 'Rasta', 'Sabdhan',
       'Sajano', 'Shasti', 'Shokti', 'Shorto', 'Shotru', 'Sokal', 'Soman',
       'Somossha', 'Somoy', 'Songbad', 'Sonkirno', 'Sosta', 'Table',
  

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass


In [ ]:
import os

# NEW Read images from folder
image_folder = 'FInal_Dataset'
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')]
x=0
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                image = cv2.imread(image_files[sequence+x])

                # Make detections
                image, results = mediapipe_detection(image, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                     # Resize image to fit screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(100)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                      # Resize image to fit screen
                    cv2.imshow('OpenCV Feed', image)

                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        x+=30
    cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

# Preprocess and Create Labels and Features

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'Ajebaje': 0,
 'Akash': 1,
 'Alada': 2,
 'Allah': 3,
 'Asha': 4,
 'Bakko': 5,
 'Bank': 6,
 'Bari': 7,
 'Bebsha': 8,
 'Bepar': 9,
 'Beyam': 10,
 'Bhromon': 11,
 'Bibaho': 12,
 'Biggan': 13,
 'Biruddhe': 14,
 'Bisoy': 15,
 'Boi': 16,
 'Boka': 17,
 'Bristi': 18,
 'Camera': 19,
 'Cha': 20,
 'Chaowa': 21,
 'Churanto': 22,
 'Dam': 23,
 'Daraw': 24,
 'Dawat': 25,
 'Dharona': 26,
 'Dhowa': 27,
 'Dokandar': 28,
 'Dol': 29,
 'Dowa_Kora': 30,
 'Druto': 31,
 'Dupur': 32,
 'Durgondho': 33,
 'Ful': 34,
 'Gari': 35,
 'Ghi': 36,
 'Ghori': 37,
 'Ghosito_Howa': 38,
 'Ghumano': 39,
 'Haat': 40,
 'Hashi': 41,
 'Hassokor': 42,
 'Injection': 43,
 'Jailkhana': 44,
 'Jinish': 45,
 'Jogajog': 46,
 'Kachi': 47,
 'Kapor': 48,
 'Kashi': 49,
 'Khaowa': 50,
 'Khoma': 51,
 'Klanto': 52,
 'Kukur': 53,
 'Mach': 54,
 'Matha': 55,
 'Matha_Betha': 56,
 'Mongol': 57,
 'Moyla': 58,
 'Name': 59,
 'Norachora': 60,
 'Ojon': 61,
 'Onusoron': 62,
 'Opomanjonok': 63,
 'Osushtho': 64,
 'Oushodh': 65,
 'Petuk': 66,
 'Phone': 67,


In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences)

In [ ]:
x = np.array(sequences)

In [ ]:
x.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.40)

# 7. Build and Train LSTM Neural network

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [14]:
log_dir = os.path.join('logs2')
tb_callback = TensorBoard(log_dir = log_dir)

In [15]:

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [ ]:
model.fit(x_train,y_train, epochs= 1000, callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
model

In [24]:
res = model.predict(x_test)

NameError: name 'x_test' is not defined

In [ ]:
actions[np.argmax(res[0])]

In [ ]:
actions[np.argmax(y_test[0])]

In [ ]:
model.save('bangla.h5')

In [16]:
model.load_weights('bangla.h5')

In [18]:
import numpy as np

# Save x_train and y_train using numpy
# np.save('x_train.npy', x_train)
# np.save('y_train.npy', y_train)

# Load x_train and y_train back from numpy files
loaded_x_train = np.load('x_train.npy')
loaded_y_train = np.load('y_train.npy')


In [19]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [20]:
yhat = model.predict(loaded_x_train )

58/58 [==============================] - 2s 26ms/step


In [21]:
ytrue = np.argmax(loaded_y_train ,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

In [22]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[1815,    0],
        [   0,   21]],

       [[1818,    0],
        [   0,   18]],

       [[1815,    0],
        [   4,   17]],

       [[1819,    0],
        [   0,   17]],

       [[1814,    1],
        [   0,   21]],

       [[1815,    0],
        [   0,   21]],

       [[1818,    0],
        [   0,   18]],

       [[1818,    1],
        [   0,   17]],

       [[1822,    0],
        [   0,   14]],

       [[1819,    1],
        [   4,   12]],

       [[1820,    1],
        [   1,   14]],

       [[1824,    0],
        [   0,   12]],

       [[1813,    0],
        [   0,   23]],

       [[1819,    0],
        [   0,   17]],

       [[1814,    0],
        [   0,   22]],

       [[1816,    1],
        [   0,   19]],

       [[1817,    1],
        [   0,   18]],

       [[1818,    0],
        [   0,   18]],

       [[1815,    3],
        [   0,   18]],

       [[1818,    0],
        [   0,   18]],

       [[1817,    0],
        [   0,   19]],

       [[1814,    0],
        [   

In [23]:
print(f"Accuracy: {accuracy_score(ytrue, yhat) * 100} %")

Accuracy: 96.18736383442265 %


In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    return output_frame

In [25]:
Language = {'Ajebaje': 'আজেবাজে', 'Akash': 'আকাশ', 'Alada': 'আলাদা', 'Allah': 'আল্লাহ', 'Asha': 'আশা', 'Bakko': 'বাক্য', 'Bank': 'ব্যাংক', 'Bari': 'বাড়ি', 'Bebsha': 'ব্যবসা', 'Bepar': 'ব্যাপার', 'Beyam': 'ব্যায়াম', 'Bhromon': 'ভ্রমণ', 'Bibaho': 'বিবাহ', 'Biggan': 'বিজ্ঞান', 'Biruddhe': 'বিরুদ্ধে', 'Bisoy': 'বিষয়', 'Boi': 'বই', 'Boka': 'বকা', 'Bristi': 'বৃষ্টি', 'Camera': 'ক্যামেরা', 'Cha': 'চা', 'Chaowa': 'চাওয়া', 'Churanto': 'চূড়ান্ত', 'Dam': 'দাম', 'Daraw': 'দাঁড়াও', 'Dawat': 'দাওয়াত', 'Dharona': 'ধারণা', 'Dhowa': 'ধোঁয়া', 'Dokandar': 'দোকানদার', 'Dol': 'দল', 'Dowa_Kora': 'দোয়া_করা', 'Druto': 'দ্রুত', 'Dupur': 'দুপুর', 'Durgondho': 'দুর্গন্ধ', 'Ful': 'ফুল', 'Gari': 'গাড়ি', 'Ghi': 'ঘি', 'Ghori': 'ঘড়ি', 'Ghosito_Howa': 'ঘোষিত_হওয়া', 'Ghumano': 'ঘুমানো', 'Haat': 'হাত', 'Hashi': 'হাসি', 'Hassokor': 'হাস্যকর', 'Injection': 'ইনজেকশন', 'Jailkhana': 'জেলখানা', 'Jinish': 'জিনিস', 'Jogajog': 'যোগাযোগ', 'Kachi': 'কাঁচি', 'Kapor': 'কাপড়', 'Kashi': 'কাশি', 'Khaowa': 'খাওয়া', 'Khoma': 'ক্ষমা', 'Klanto': 'ক্লান্ত', 'Kukur': 'কুকুর', 'Mach': 'মাছ', 'Matha': 'মাথা', 'Matha_Betha': 'মাথা_ব্যথা', 'Mongol': 'মঙ্গল', 'Moyla': 'ময়লা', 'Name': 'নাম', 'Norachora': 'নড়াচড়া', 'Ojon': 'ওজন', 'Onusoron': 'অনুসরণ', 'Opomanjonok': 'অপমানজনক', 'Osushtho': 'অসুস্থ', 'Oushodh': 'ঔষধ', 'Petuk': 'পেটুক', 'Phone': 'ফোন', 'Pochondo': 'পছন্দ', 'Porikkha': 'পরীক্ষা', 'Poriskar': 'পরিস্কার', 'Prostut': 'প্রস্তুত', 'Protarona': 'প্রতারণা', 'Raat': 'রাত', 'Rajdhani': 'রাজধানী', 'Rasta': 'রাস্তা', 'Sabdhan': 'সাবধান', 'Sajano': 'সাজানো', 'Shasti': 'শাস্তি', 'Shokti': 'শক্তি', 'Shorto': 'শর্ত', 'Shotru': 'শত্রু', 'Sokal': 'সকাল', 'Soman': 'সমান', 'Somossha': 'সমস্যা', 'Somoy': 'সময়', 'Songbad': 'সংবাদ', 'Sonkirno': 'সংকীর্ণ', 'Sosta': 'সস্তা', 'Table': 'টেবিল', 'Taka': 'টাকা', 'Tamasha': 'তামাশা', 'Tapmatra': 'তাপমাত্রা', 'Tarikh': 'তারিখ', 'Touri_Kora': 'তৈরি_করা', 'Tumi': 'তুমি', 'Unnoto': 'উন্নত', 'Upor': 'উপর', 'Vaggo': 'ভাগ্য', 'Valo': 'ভালো', 'Vari': 'ভারী', 'Vule_jawa': 'ভুলে_যাওয়া'}


In [26]:
sequence = []
sentence = []
threshold = 0.3

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(Language[actions[np.argmax(res)]])
            cv2.waitKey(500)


        #3. Viz logic
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
            # image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

KeyboardInterrupt: 

In [ ]:
cap.release()
cv2.destroyAllWindows()